#### Data/Text Mining Project


Course: CISD 43<br>
Professor: Sohair Zaki<br>
Student: Fiona Xu and Jack Chen<br>
<br>
Purpose:<br>
- hello world
<br>

Details:<br>
1. Load libraries, data cleaning<br>
2. Exploratory Data Analysis (EDA)<br>
3. Deep Learning<br>
4. Conclusion<br>
5. References<br>

#### Load/Cleaning Data

In [ ]:
# import our libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

#### Exploratory Data Analysis (EDA)

#### Deep Learning

#### Conclusion


#### References


----------
CIS43 Project Link<br>
https://colab.research.google.com/drive/1OS0i9lEitkL7PTE7TpLuWVoqkjS0BPYC
<br>

Load local file onto colab<br>
https://www.youtube.com/watch?v=6HFlwqK3oeo
<br>
<br>

----------
Inflation Data<br>
https://www.usinflationcalculator.com/inflation/current-inflation-rates/#:~:text=The%20annual%20inflation%20rate%20for,at%208%3A30%20a.m.%20ET.
<br>
<br>

CO2 Emissions<br>
https://ourworldindata.org/co2-emissions
<br>
<br>
Kaggle Data Temperature Change<br>
https://www.kaggle.com/datasets/sevgisarac/temperature-change?select=Environment_Temperature_change_E_All_Data_NOFLAG.csv
<br>
<br>